In [ ]:
import numpy as np
import scipy.sparse
import random

def empty_matrix(n, m, format):
    Matrix = scipy.sparse.__dict__[format + "_matrix"]
    return Matrix((n, m))


def identity_matrix(n, format):
    return scipy.sparse.identity(n, format=format)


def ascending_vector(n):
    return np.array(list(range(1, n + 1)))

def generate_big_matrix(n, p, format):
    return scipy.sparse.random(n, n, p, format=format)

In [ ]:
def generate_pre_matrix(n):
    A = empty_matrix(n, n, "lil")
    for i in range(n):
        for j in range(n):
            A[i, j] = random.randint(-4, 0)

    return A

def generate_diagonal_domination_matrix(n, k):
    a = generate_pre_matrix(n)
    A_k = empty_matrix(n, n, "lil")

    for i in range(n):
        t1 = -sum(a[i, ind] for ind in range(i))
        t2 = -sum(a[i, ind] for ind in range(i + 1, n))
        t = t1 + t2
        for j in range(n):
            if i != j:
                A_k[i, j] = a[i, j]
            else:
                A_k[i, j] = t + pow(10.0, -k)

    A_k = A_k.tocsr()
    return A_k


def generate_hilbert_matrix(n):
    A_k = empty_matrix(n, n, "lil")
    for i in range(n):
        for j in range(n):
            A_k[i, j] = 1.0 / (i + j + 1.0)

    return A_k.tocsr()

In [ ]:
def lil_row_product(matrix1, matrix2, row_idx1, row_idx2):
    row1 = matrix1.rows[row_idx1]
    row2 = matrix2.rows[row_idx2]
    data1 = matrix1.data[row_idx1]
    data2 = matrix2.data[row_idx2]

    res = 0
    i1 = i2 = 0

    while i1 < len(row1) and i2 < len(row2):
        col_idx1 = row1[i1]
        col_idx2 = row2[i2]

        if col_idx1 == col_idx2:
            res += data1[i1] * data2[i2]
            i1 += 1
        elif col_idx1 < col_idx2:
            i1 += 1
        else:
            i2 += 1

    return res


def csr_row_iter(csr, row_idx):
    row_len = csr.shape[1]
    data_start = csr.indptr[row_idx]
    data_end = csr.indptr[row_idx + 1] if row_idx + 1 < csr.shape[0] else len(csr.data)

    j = 0

    for data_idx in range(data_start, data_end):
        col = csr.indices[data_idx]
        val = csr.data[data_idx]

        while j < col:
            yield 0
            j += 1
        
        yield val
        j += 1
    
    for _ in range(j, row_len):
        yield 0


def lu_decomposition(A):
    N = A.shape[0]
    L = identity_matrix(N, "lil")
    U = empty_matrix(N, N, "lil")  # транспонированная

    for i in range(N):
        for j, a in zip(range(N), csr_row_iter(A, i)):
            num = a - lil_row_product(U, L, j, i)

            if i <= j:
                U[j, i] = num
            else:
                L[i, j] = num / U[j, j]

    return L.tocsr(), U.transpose().tocsr()


def lower_trivial_system_solution(A, b):
    x = np.zeros(len(b))
    x[0] = b[0]

    for i in range(1, len(b)):
        x[i] = b[i] - A[i] * x

    return x


def upper_trivial_system_solution(A, b):
    N = len(b)
    x = np.zeros(N)
    x[-1] = b[-1] / A[-1, -1]

    for i in reversed(range(N - 1)):
        x[i] = (b[i] - A[i] * x) / A[i, i]

    return x


def system_solution(A, b):
    L, U = lu_decomposition(A)
    y = lower_trivial_system_solution(L, b)
    x = upper_trivial_system_solution(U, y)
    return x

In [ ]:
def system_solution_matrix(A, B):
    N = A.shape[0]
    X = empty_matrix(N, N, "lil")
    for i in range(N):
        X[i] = system_solution(A, B.getcol(i).toarray())
    return X.tocsr().transpose()


def inverse_matrix(A):
    return system_solution_matrix(A, identity_matrix(A.shape[0], "csr"))

In [ ]:
def seidel_row_vec_product(csr, vec1, vec2, row_idx):
    start_idx = csr.indptr[row_idx]
    end_idx = csr.indptr[row_idx + 1] if row_idx + 1 < csr.shape[0] else len(csr.data)

    curr_vec = vec1
    product = 0
    diag_value = 0

    for i in range(start_idx, end_idx):
        col_idx = csr.indices[i]
        val = csr.data[i]

        if col_idx >= row_idx:
            curr_vec = vec2

        if col_idx == row_idx:
            diag_value = val
            continue

        product += val * curr_vec[col_idx]

    return product, diag_value

def ostanov(x, x_new, eps):
    norma = 0
    for i in range(len(x)):
        norma += (x[i] - x_new[i]) ** 2
    return norma ** 0.5 <= eps

def seidel_method(A, b, eps=1e-6, max_iter=250):
    n = A.shape[0]
    x = np.array(b)

    for _ in range(max_iter):
        x_new = np.zeros(n)
        for i in range(n):
            product, diag_value = seidel_row_vec_product(A, x_new, x, i)
            x_new[i] = (b[i] - product) / diag_value

        if ostanov(x, x_new, eps):
            break

        x = x_new
    else:
        print(f"Seidel method diverges with {_}")
        return x

    print(f"Seidel method converges with {_}")
    return x

In [ ]:
def process_seidel_row_vec_product(csr, first_vector, second_vector, row_index):
    begin_index = csr.indptr[row_index]
    end_index = csr.indptr[row_index + 1] if row_index + 1 < csr.shape[0] else len(csr.data)

    curr_vector = first_vector
    product = 0
    diagonal_value = 0
    counter = 0

    for i in range(begin_index, end_index):
        column_index = csr.indices[i]
        value = csr.data[i]

        if column_index >= row_index:
            curr_vector = second_vector

        if column_index == row_index:
            diagonal_value = value
            continue

        product += value * curr_vector[column_index]

    return product, diagonal_value


def seidel_method(A, b, eps=1e-6, maximum_iterations_number=2500):
    n = A.shape[0]
    x = np.array(b)

    for _ in range(maximum_iterations_number):
        x_new = np.zeros(n)
        for i in range(n):
            product, diagonal_value = process_seidel_row_vec_product(A, x_new, x, i)
            x_new[i] = (b[i] - product) / diagonal_value

        if np.allclose(x, x_new, rtol=eps):
            break

        x = x_new
    else:
        raise RuntimeError("Seidel method diverges!")
    print(f"Seidel method converges with {_}")
    return x

In [ ]:
first_matrix = [[2, 1, 1], [0, 3, 2], [0, 0, 7]]
first_vector = [9, 8, 7]
first_expected_answer = [3, 2, 1]

second_matrix = [[5, 1, 1], [1, 6, 2], [0, 1, 7]]
second_vector = [7, 13, 2]
second_expected_answer = [1, 2, 0]

first_csr_matrix = scipy.sparse.csr_matrix(first_matrix)
first_actual_answer = seidel_method(first_csr_matrix, first_vector, 10e-3)
second_csr_matrix = scipy.sparse.csr_matrix(second_matrix)
second_actual_answer = seidel_method(second_csr_matrix, second_vector, 10e-3)

print(np.isclose(first_actual_answer, first_expected_answer).all())
print(np.isclose(second_actual_answer, second_expected_answer).all())

Seidel method converges with 3
Seidel method converges with 7
True
True


In [ ]:
def seidel(A, b, eps=0.001):
    n = A.shape[0]
    x = np.zeros(n)  # zero vector

    max_iter = 2500
    for _ in range(max_iter):
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i, j] * x_new[j] for j in range(i))
            s2 = sum(A[i, j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i, i]

        if ostanov(x, x_new, eps):
            break
        x = x_new
    else:
        print(f"Seidel method diverges with {_}")
        return x

    print(f"Seidel method converges with {_}")
    return x

In [ ]:
from scipy.sparse import linalg

def generate_nonsingular_matrix(n, p):
    matrix = generate_big_matrix(n, p, "lil")
    for i in range(n):
        matrix[i, i] += 1000
    return matrix


def generate_diagonal_domination_matrix(a, k):
    n = len(a)
    A_k = empty_matrix(n, n, "lil").tolil()

    for i in range(n):
        t1 = -sum(a[i][k] for k in range(i))
        t2 = -sum(a[i][k] for k in range(i + 1, n))
        t = t1 + t2
        for j in range(n):
            if i == j:
                A_k[i, j] = t
            else:
                A_k[i, j] = t + pow(10.0, -k)

    A_k = A_k.tocsr()
    return A_k


def generate_hilbert_matrix(k):
    A_k = empty_matrix(k, k, "lil").tolil()
    for i in range(k):
        for j in range(k):
            A_k[i, j] = 1.0 / (i + j + 1.0)

    return A_k.tocsr()


def generate_test_data(n, p=0.3):
    matrix = generate_nonsingular_matrix(n, p).tocsr()
    x = ascending_vector(n)
    vector = matrix * x
    return matrix, vector


matrix, vector = generate_test_data(10000, p=0.01)

In [ ]:
for n in [5, 10, 50, 100, 1000, 5000]:
    matrix, vector = generate_test_data(n, p=0.01)
    seidel_method(matrix, vector, 0.001, 250)

Seidel method converges with 1
Seidel method converges with 1
Seidel method converges with 2
Seidel method converges with 3
Seidel method converges with 4
Seidel method converges with 5
